# Parallel computing and reusing realizations

In this example, we demonstrate two features of the library that help reduce calculation time:
* Parallel computing – a method of performing multiple calculations simultaneously to improve efficiency.
* Reusing realizations – avoiding redundant computations by utilizing previously generated realizations.

We apply these features to calculate the probability of levee failure due to wave overtopping.

First, we import the necessary packages:

In [1]:
from probabilistic_library import ReliabilityProject, DistributionType, ReliabilityMethod
import time

We consider a limit state function for wave overtopping (which we have artificially slowed down):

In [2]:
from utils.models import ZFunctionOvertopping

And the following reliability project, in which we apply the `crude_monte_carlo` method:

In [3]:
def define_project():

    project = ReliabilityProject()
    project.model = ZFunctionOvertopping.z_sleep
    
    project.variables["h"].distribution = DistributionType.log_normal
    project.variables["h"].mean = 1.5
    project.variables["h"].deviation = 0.05

    project.variables["hm0"].distribution = DistributionType.log_normal
    project.variables["hm0"].mean = 1.5
    project.variables["hm0"].deviation = 0.25

    project.variables["tm10"].distribution = DistributionType.log_normal
    project.variables["tm10"].mean = 3
    project.variables["tm10"].deviation = 0.5

    project.variables["wave_direction"].distribution = DistributionType.deterministic
    project.variables["wave_direction"].mean = 0.0

    project.variables["dike_normal"].distribution = DistributionType.deterministic
    project.variables["dike_normal"].mean = 0.0

    project.variables["y_crest"].distribution = DistributionType.deterministic
    project.variables["y_crest"].mean = 6.0

    project.variables["q_crit"].distribution = DistributionType.log_normal
    project.variables["q_crit"].mean = 0.001
    project.variables["q_crit"].deviation = 0.01

    project.settings.reliability_method = ReliabilityMethod.crude_monte_carlo
    project.settings.minimum_samples = 1000
    project.settings.maximum_samples = 1000
    project.settings.variation_coefficient = 0.02

    return project

### Parallel computing

If not specified, the calculations are performed on a single processor. To utilize multiple processors, we adjust the setting `project.settings.max_parallel_processes`. 

With the following code, we analyze the effect of parallel computing on the calculation time:

In [4]:
max_parallel_processes = [1, 4, 8]

for val in max_parallel_processes:

    project = define_project()
    project.settings.max_parallel_processes = val

    t = time.time()
    project.run()
    elapsed = time.time() - t
    
    print(f"Max parallel processes: {val}")
    print(f"Time elapsed: {elapsed} seconds")

    project.design_point.print()

Max parallel processes: 1
Time elapsed: 108.83074307441711 seconds
Reliability:
 Reliability index = 1.2265281200366105
 Probability of failure = 0.10999999999999993
 Convergence = 0.08994948077064754 (not converged)
 Model runs = 1001
Alpha values:
 self: alpha = 0.0, x = 0.0
 h: alpha = -0.04342108406821106, x = 1.5018303649169757
 hm0: alpha = -0.3715913012248334, x = 1.595531805516477
 tm10: alpha = -0.6051390499098047, x = 3.3460139874125105
 wave_direction: alpha = 0.0, x = 0.0
 dike_normal: alpha = 0.0, x = 0.0
 y_crest: alpha = 0.0, x = 6.0
 q_crit: alpha = 0.7027383898625686, x = 1.5619634827850072e-05


Max parallel processes: 4
Time elapsed: 26.060221195220947 seconds
Reliability:
 Reliability index = 1.2265281200366105
 Probability of failure = 0.10999999999999993
 Convergence = 0.08994948077064754 (not converged)
 Model runs = 1001
Alpha values:
 self: alpha = 0.0, x = 0.0
 h: alpha = -0.04342108406821106, x = 1.5018303649169757
 hm0: alpha = -0.3715913012248334, x = 1.595

### Reusing realizations

Another useful feature of the library is the ability to reuse realizations - this is possible if the project is not redefined when it is run again.

This functionality is particularly valuable in sensitivity analyses (if multiple model outputs are required). It is also beneficial when applying the Crude Monte Carlo method in reliability analysis. For instance, if a calculation is first performed with $200$ samples and later extended to $250$ samples, the initial $200$ samples are reused in the second run, avoiding unnecessary recomputation. 

This is demonstrated in the following example:

In [5]:
project = define_project()
samples = [200, 200, 250]
run_message = ["Initial run", "Run repeated", "Additional 50 runs"]

for id in range(3):
    project.settings.minimum_samples = samples[id]
    project.settings.maximum_samples = samples[id]

    t = time.time()
    project.run()
    elapsed = time.time() - t

    print(f"{run_message[id]}")
    print(f"Time elapsed: {elapsed} seconds")

    project.design_point.print()

Initial run
Time elapsed: 20.403820276260376 seconds
Reliability:
 Reliability index = 1.2535654384704509
 Probability of failure = 0.10499999999999997
 Convergence = 0.20644381225662256 (not converged)
 Model runs = 201
Alpha values:
 self: alpha = 0.0, x = 0.0
 h: alpha = 0.01244189948906268, x = 1.498388375417174
 hm0: alpha = -0.46218581222364535, x = 1.6285145936446288
 tm10: alpha = -0.5702735946213111, x = 3.3309032020254223
 wave_direction: alpha = 0.0, x = 0.0
 dike_normal: alpha = 0.0, x = 0.0
 y_crest: alpha = 0.0, x = 6.0
 q_crit: alpha = 0.6789826959458994, x = 1.5985554061827512e-05


Run repeated
Time elapsed: 0.008946657180786133 seconds
Reliability:
 Reliability index = 1.2535654384704509
 Probability of failure = 0.10499999999999997
 Convergence = 0.20644381225662256 (not converged)
 Model runs = 0
Alpha values:
 self: alpha = 0.0, x = 0.0
 h: alpha = 0.01244189948906268, x = 1.498388375417174
 hm0: alpha = -0.46218581222364535, x = 1.6285145936446288
 tm10: alpha = -